<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/EA_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math

# --- CONFIGURATION ---
POINTS = 10         # Resolution of our wing profile
POP_SIZE = 50
GENERATIONS = 100
MUTATION_RATE = 0.1

def calculate_fitness(airfoil):
    """
    Simulated Aerodynamics:
    1. Reward 'Smoothness' (low drag)
    2. Reward 'Camber' (curvature for lift)
    3. Penalize 'Extreme Spikes' (turbulence)
    """
    lift = sum(airfoil) * 0.5  # Area under curve
    drag = 0
    for i in range(len(airfoil) - 1):
        drag += abs(airfoil[i+1] - airfoil[i]) ** 2 # Sharp changes = high drag

    # Fitness = Lift / Drag (avoiding division by zero)
    return lift / (drag + 0.01)

def create_individual():
    # Start with a random set of heights between 0 and 1
    return [random.uniform(0, 1) for _ in range(POINTS)]

def mutate(airfoil):
    return [h + random.uniform(-0.1, 0.1) if random.random() < MUTATION_RATE else h for h in airfoil]

def crossover(p1, p2):
    split = random.randint(1, POINTS - 1)
    return p1[:split] + p2[split:]

# --- EVOLUTIONARY LOOP ---
population = [create_individual() for _ in range(POP_SIZE)]

for gen in range(GENERATIONS):
    # Score and Sort
    population = sorted(population, key=calculate_fitness, reverse=True)
    best = population[0]

    if gen % 10 == 0:
        visual = "".join(["#" if h > 0.5 else "-" for h in best])
        print(f"Gen {gen} | Fitness: {calculate_fitness(best):.2f} | Shape: {visual}")

    # Breeding (Top 20% survive)
    next_gen = population[:10]
    while len(next_gen) < POP_SIZE:
        parent1, parent2 = random.sample(population[:20], 2)
        child = crossover(parent1, parent2)
        child = mutate(child)
        next_gen.append(child)

    population = next_gen

print("\nFinal Optimized Wing Profile (Heights):")
print([round(h, 2) for h in population[0]])

Gen 0 | Fitness: 4.08 | Shape: ---######-
Gen 10 | Fitness: 36.33 | Shape: #--#######
Gen 20 | Fitness: 120.84 | Shape: --########
Gen 30 | Fitness: 169.33 | Shape: ##########
Gen 40 | Fitness: 180.46 | Shape: ##########
Gen 50 | Fitness: 190.37 | Shape: ##########
Gen 60 | Fitness: 195.42 | Shape: ##########
Gen 70 | Fitness: 211.71 | Shape: ##########
Gen 80 | Fitness: 223.80 | Shape: ##########
Gen 90 | Fitness: 226.09 | Shape: ##########

Final Optimized Wing Profile (Heights):
[0.54, 0.54, 0.56, 0.58, 0.61, 0.63, 0.66, 0.68, 0.7, 0.7]


In [2]:
import random

# --- CONFIGURATION ---
POINTS = 10
POP_SIZE = 60
GENERATIONS = 150
MUTATION_RATE = 0.1
FUEL_CAPACITY_REQ = 4.5  # The "Constraint": Sum of heights must be at least this

def calculate_fitness(airfoil):
    # 1. Aerodynamic Efficiency (same as before)
    lift = sum(airfoil) * 0.5
    drag = sum(abs(airfoil[i+1] - airfoil[i])**2 for i in range(len(airfoil)-1)) + 0.01
    efficiency = lift / drag

    # 2. The Constraint Penalty
    volume = sum(airfoil)
    if volume < FUEL_CAPACITY_REQ:
        # Penalize designs that are too thin to hold fuel
        efficiency *= 0.1

    return efficiency

def create_individual():
    return [random.uniform(0.1, 1.0) for _ in range(POINTS)]

def mutate(airfoil):
    return [max(0.05, min(1.0, h + random.uniform(-0.1, 0.1))) if random.random() < MUTATION_RATE else h for h in airfoil]

def crossover(p1, p2):
    split = random.randint(1, POINTS - 1)
    return p1[:split] + p2[split:]

# --- EVOLUTIONARY LOOP ---
population = [create_individual() for _ in range(POP_SIZE)]

for gen in range(GENERATIONS):
    population = sorted(population, key=calculate_fitness, reverse=True)
    best = population[0]

    if gen % 20 == 0:
        vol = sum(best)
        print(f"Gen {gen:03} | Fitness: {calculate_fitness(best):.2f} | Volume: {vol:.2f}")

    next_gen = population[:12] # Elitism
    while len(next_gen) < POP_SIZE:
        p1, p2 = random.sample(population[:25], 2)
        child = mutate(crossover(p1, p2))
        next_gen.append(child)
    population = next_gen

print("\nFinal 'Structural' Wing Profile:")
print([round(h, 2) for h in population[0]])

Gen 000 | Fitness: 8.12 | Volume: 7.30
Gen 020 | Fitness: 225.43 | Volume: 6.51
Gen 040 | Fitness: 275.66 | Volume: 6.51
Gen 060 | Fitness: 287.69 | Volume: 6.50
Gen 080 | Fitness: 290.32 | Volume: 6.50
Gen 100 | Fitness: 295.54 | Volume: 6.52
Gen 120 | Fitness: 300.15 | Volume: 6.53
Gen 140 | Fitness: 303.69 | Volume: 6.54

Final 'Structural' Wing Profile:
[0.62, 0.63, 0.63, 0.64, 0.64, 0.66, 0.67, 0.68, 0.69, 0.69]


## EA AND WM

In [4]:
import numpy as np

# --- CONFIGURATION ---
NUM_MODELS = 10     # Simulating 10 "Cortical Columns"
POP_SIZE = 50       # Individuals in the EA population
GENERATIONS = 50
MUTATION_RATE = 0.05

# 1. THE WORLD MODELS (Cortical Columns)
class CorticalColumn:
    def __init__(self, id):
        self.id = id
        # Random bias simulates different sensory perspectives or errors
        self.bias = np.random.uniform(-0.5, 0.5)

    def predict(self, true_value):
        # Models are noisy; some are better than others
        noise = np.random.normal(0, 0.2)
        return true_value + self.bias + noise

models = [CorticalColumn(i) for i in range(NUM_MODELS)]

# 2. THE EA GENOME
def create_individual():
    weights = np.random.rand(NUM_MODELS)
    return weights / weights.sum()  # Normalized "trust" vector

def get_fitness(weights, test_cases):
    total_error = 0
    for true_val in test_cases:
        predictions = np.array([m.predict(true_val) for m in models])
        consensus = np.dot(weights, predictions)  # Weighted consensus vote
        total_error += abs(true_val - consensus)
    return 1 / (total_error + 0.01)  # Minimize prediction error

# 3. EVOLUTIONARY LOOP
population = [create_individual() for _ in range(POP_SIZE)]
test_data = np.linspace(-10, 10, 20)  # Assessment environment

print(f"{'Gen':<5} | {'Best Fitness':<15} | {'Top Weights (First 5)'}")
print("-" * 65)

for gen in range(GENERATIONS):
    # Sort by fitness (descending)
    population = sorted(population, key=lambda ind: get_fitness(ind, test_data), reverse=True)
    best_ind = population[0]

    if gen % 10 == 0 or gen == GENERATIONS - 1:
        weight_str = " ".join([f"{w:.2f}" for w in best_ind[:5]]) + " ..."
        print(f"{gen:<5} | {get_fitness(best_ind, test_data):<15.4f} | {weight_str}")

    # Breeding (Elitism + Selection + Crossover + Mutation)
    next_gen = population[:10]  # Carry over top performers
    while len(next_gen) < POP_SIZE:
        # CORRECTED: Select indices rather than the objects themselves
        idx1, idx2 = np.random.choice(len(population[:20]), 2, replace=False)
        p1, p2 = population[idx1], population[idx2]

        # Simple Crossover
        child = (p1 + p2) / 2

        # Mutation: Add slight variation to trust levels
        if np.random.rand() < MUTATION_RATE:
            child += np.random.normal(0, 0.1, NUM_MODELS)

        # Ensure weights remain positive and normalized
        child = np.clip(child, 1e-6, 1.0)
        next_gen.append(child / child.sum())

    population = next_gen

print("\nFinal Optimized Trust Distribution:")
print(f"Weights for 10 Parallel Models: {[round(w, 2) for w in population[0]]}")

Gen   | Best Fitness    | Top Weights (First 5)
-----------------------------------------------------------------
0     | 0.6018          | 0.11 0.06 0.00 0.17 0.08 ...
10    | 0.8529          | 0.13 0.09 0.11 0.10 0.09 ...
20    | 0.9108          | 0.13 0.10 0.12 0.10 0.09 ...
30    | 0.9144          | 0.13 0.09 0.12 0.10 0.09 ...
40    | 1.1167          | 0.13 0.09 0.12 0.09 0.09 ...
49    | 1.5984          | 0.13 0.09 0.12 0.09 0.09 ...

Final Optimized Trust Distribution:
Weights for 10 Parallel Models: [np.float64(0.13), np.float64(0.09), np.float64(0.12), np.float64(0.09), np.float64(0.09), np.float64(0.1), np.float64(0.08), np.float64(0.07), np.float64(0.09), np.float64(0.13)]
